In [1]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

print('Packages installed.')

     |████████████████████████████████| 102kB 8.6MB/s ta 0:00:011
     |████████████████████████████████| 102kB 7.9MB/s ta 0:00:011
Packages installed.


In [2]:
pip install BeautifulSoup4


The following command must be run outside of the IPython shell:

    $ pip install BeautifulSoup4

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [5]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [6]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [7]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)

Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)


TypeError: 'NoneType' object is not subscriptable

In [8]:
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A,Not assigned,Not assigned,43.648690,-79.385440
1,M2A,Not assigned,Not assigned,43.648690,-79.385440
2,M3A,North York,Parkwoods,43.752935,-79.335641
3,M4A,North York,Victoria Village,43.728102,-79.311890
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


In [9]:
neighbors.shape

(7, 5)

In [10]:
VERSION = '20200611' # Foursquare API version

neighborhood_name = neighbors.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_latitude = neighbors.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbors.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

# url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={20200611}&ll={43.67635739999999},{79.2930312}&radius={}&limit={}'.format(
#     CLIENT_ID, 
#     CLIENT_SECRET, 
#     VERSION, 
#     43.67635739999999, 
#     79.2930312, 
#     radius, 
#     LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore?&client_id=&client_secret=&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

results = requests.get(url).json()
results

Latitude and longitude values of Not assigned
 are 43.648690000000045, -79.38543999999996.


{'meta': {'code': 200, 'requestId': '5ee155f5cf9b553d6825d8ea'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [14]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
#         url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#             CLIENT_ID, 
#             CLIENT_SECRET, 
#             VERSION, 
#             lat, 
#             lng, 
#             radius, 
#             LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

Not assigned

Not assigned

Parkwoods

Victoria Village

Regent Park, Harbourfront

Lawrence Manor, Lawrence Heights

Queen's Park, Ontario Provincial Government



In [16]:
print(toronto_venues.shape)
toronto_venues.head()

(28, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Not assigned,43.64869,-79.38544,Glen Manor Ravine,43.676821,-79.293942,Trail
1,Not assigned,43.64869,-79.38544,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,Not assigned,43.64869,-79.38544,Grover Pub and Grub,43.679181,-79.297215,Pub
3,Not assigned,43.64869,-79.38544,Upper Beaches,43.680563,-79.292869,Neighborhood
4,Not assigned,43.64869,-79.38544,Glen Manor Ravine,43.676821,-79.293942,Trail


In [17]:
# check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Lawrence Manor, Lawrence Heights",4,4,4,4,4,4
Not assigned,8,8,8,8,8,8
Parkwoods,4,4,4,4,4,4
"Queen's Park, Ontario Provincial Government",4,4,4,4,4,4
"Regent Park, Harbourfront",4,4,4,4,4,4
Victoria Village,4,4,4,4,4,4


In [18]:
# find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 4 uniques categories.


In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Trail,Health Food Store,Neighborhood,Pub
0,1,0,Not assigned,0
1,0,1,Not assigned,0
2,0,0,Not assigned,1
3,0,0,Not assigned,0
4,1,0,Not assigned,0


In [20]:
# examine the new dataframe size.

toronto_onehot.shape

(28, 4)

In [21]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Trail,Health Food Store,Pub
0,"Lawrence Manor, Lawrence Heights",0.25,0.25,0.25
1,Not assigned,0.25,0.25,0.25
2,Parkwoods,0.25,0.25,0.25
3,"Queen's Park, Ontario Provincial Government",0.25,0.25,0.25
4,"Regent Park, Harbourfront",0.25,0.25,0.25
5,Victoria Village,0.25,0.25,0.25


In [22]:
# confirm the new size

toronto_grouped.shape

(6, 4)

In [23]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Lawrence Manor, Lawrence Heights
----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


----Not assigned
----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


----Parkwoods
----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


----Queen's Park, Ontario Provincial Government
----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


----Regent Park, Harbourfront
----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


----Victoria Village
----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25




In [24]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,"Lawrence Manor, Lawrence Heights",Pub,Health Food Store,Trail
1,Not assigned,Pub,Health Food Store,Trail
2,Parkwoods,Pub,Health Food Store,Trail
3,"Queen's Park, Ontario Provincial Government",Pub,Health Food Store,Trail
4,"Regent Park, Harbourfront",Pub,Health Food Store,Trail


In [26]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0], dtype=int32)

In [27]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1A,Not assigned,Not assigned,43.648690,-79.385440,0,Pub,Health Food Store,Trail
1,M2A,Not assigned,Not assigned,43.648690,-79.385440,0,Pub,Health Food Store,Trail
2,M3A,North York,Parkwoods,43.752935,-79.335641,0,Pub,Health Food Store,Trail
3,M4A,North York,Victoria Village,43.728102,-79.311890,0,Pub,Health Food Store,Trail
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,0,Pub,Health Food Store,Trail


In [28]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[43.67635739999999, -79.2930312], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [29]:
# CLUSTER 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Not assigned,0,Pub,Health Food Store,Trail
1,Not assigned,0,Pub,Health Food Store,Trail
2,North York,0,Pub,Health Food Store,Trail
3,North York,0,Pub,Health Food Store,Trail
4,Downtown Toronto,0,Pub,Health Food Store,Trail
5,North York,0,Pub,Health Food Store,Trail
6,Downtown Toronto,0,Pub,Health Food Store,Trail
